In [0]:
%sql
USE CATALOG formula1;

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW PILOTOS_VENCEDORES AS
SELECT DISTINCT
  (driverId) as driver_id,
  try_cast(position AS INT) as position_int
FROM
  formula1.bronze.results
WHERE
  try_cast(position AS INT) > 10

In [0]:
%sql
CREATE OR REPLACE TABLE formula1.silver.results AS
SELECT
  resultId AS result_id,
  raceId AS race_id,
  driverId AS driver_id,
  constructorId AS constructor_id,
  number,
  grid,
  position,
  positionText,
  CAST(positionOrder AS INT) AS position_order,
  points,
  laps,
  time,
  milliseconds,
  fastestLap,
  try_cast(rank AS INT) AS rank,
  fastestLapTime,
  fastestLapSpeed,
  --statusId,
  bronze_ingestion,
  current_timestamp AS silver_ingestion
FROM
  formula1.bronze.results
WHERE
  try_cast(driverId AS INT) IN (
    SELECT
      driver_id
    FROM
      PILOTOS_VENCEDORES
  )

In [0]:
%sql
CREATE OR REPLACE TABLE formula1.silver.drives AS
SELECT
  driverId AS driver_id,
  driverRef,
  number,
  code,
  forename,
  surname,
  dob,
  url,
  nationality,
  current_timestamp AS silver_ingestion
FROM
  formula1.bronze.drivers

In [0]:
%sql
CREATE OR REPLACE TABLE formula1.silver.constructors AS
SELECT
  constructorId AS constructor_id,
  constructorRef,
  name,
  nationality,
  url,
  current_timestamp AS silver_ingestion
FROM
  formula1.bronze.constructors

In [0]:
%sql
CREATE OR REPLACE TABLE formula1.silver.races AS
SELECT
  raceId,
  year,
  round,
  circuitId,
  name,
  date,
  time,
  url,
  fp1_date,
  fp1_time,
  fp2_date,
  fp2_time,
  fp3_date,
  fp3_time,
  quali_date,
  quali_time,
  sprint_date,
  sprint_time,
  bronze_ingestion,
  current_timestamp AS silver_ingestion
FROM
  formula1.bronze.races

In [0]:
dbutils.notebook.exit(0)

In [0]:
%sql
CREATE OR REPLACE TABLE formula1.silver.race_results AS
SELECT
  race_id,
  year,
  circuit_name,
  drivers.driver_id,
  drivers.driverRef AS driver_name,
  position,
  results.bronze_ingestion
FROM
  formula1.silver.results AS results
    INNER JOIN formula1.silver.races AS races
      ON results.race_id = races.raceId
    INNER JOIN formula1.silver.circuits AS circuits
      ON races.circuitId = circuits.circuit_id
    INNER JOIN formula1.silver.drives AS drivers
      ON results.driver_id = drivers.driver_id

In [0]:
%sql
SELECT * FROM formula1.silver.race_results
WHERE position = '1'

Databricks visualization. Run in Databricks to view.